# 제2 고지 : 자연스러운  코드로 
## STEP 17 : 메모리 관리와 순환 참조

지금까지의 구현에서는 **속도와 메모리 사용량에 전혀 신경쓰지 않았다.** 하지만 속도와 메모리의 최적화는 프로그래밍에 있어 너무나도 중요한 부분이기 때문에 이번 단계와 다음 단계를 통해 성능을 개선해보도록 한다.  
본격적인 시작에 앞서 **파이썬의 메모리 관리**에 대해 살펴보자.  
(표준으로 사용되는 파이썬 인터프리터 `CPython`을 기준으로 설명한다.)

### 17.1 메모리 관리 

파이썬은 **필요없어진 객체를 메모리에서 자동으로 삭제**한다. 그럼에도 불구하고, 잘못된 코드 작성은 `memory leak` 또는 `out of memory` 문제가 발생할 수 있다.  
일반적으로 파이썬은 다음 두가지 방식으로 메모리를 관리한다.  
1. 참조(reference)수를 세는 방식.  (=`참조 카운트`)
2. 세대(generation)을 기준으로 쓸모 없어진 객체를 회수하는 방식 (=`Garbage Collection,GC`)  
(문헌에 따라 `참조 카운트`방식도 `GC` 로 보기도 하지만 해당 책에서는 두가지를 구분하여 설명한다.)

### 17.2 참조 카운트 방식의 메모리 관리 

파이썬 메모리 관리의 기본은 `참조 카운트` 이다. 이는 간단한 구조로 이뤄져 있는데, 다음과 같이 작동한다.  
1. **모든 객체는 참조 카운트가 0으로 시작**한다.
2. **다른 객체가 참조할때마다 +1** 이 되며, **반대로 참조가 끊길때 마다 -1** 된다.
3. 1,2의 방식을 바탕으로 **객체가 더이상 필요 없어지면(=참조 카운트가 0) 메모리에서 삭제**된다.

예를 들어, 다음과 같은 경우에 참조카운트가 증가한다.  
1. 대입 연산자를 사용할 때 
2. 함수에 인수로 전달할 때
3. 컨테이너 타입 객체(리스트,튜플,클래스 등)에 추가할 때

구체적으로, 다음 예제를 통해 살펴보자.

![image](../assets/%EA%B7%B8%EB%A6%BC%2017-1.png)

```python
class obj:
    pass

a = obj() 
b = obj()
c = obj()

a.b = b 
b.c = c 

a=b=c=None 
```
하나씩 살펴보면, 
1. `a`,`b`,`c` 라는 객체를 생성했다. 
2. 그리고 a가 b를 참조하고, b가 c 를 참조한다. 즉 위의 왼쪽 그림과 같은 상황이 되었다. 
3. 이후 `a=b=c=None` 을 실행하면 위의 오른쪽 그림과 같이 된다. 이때, **a의 참조 카운트는 0, b와 c 의 참조카운트는 1**이다.  
4. **a는 즉시 메모리에서 삭제되고, a가 삭제 되었으므로 b 참조 카운트 역시 0이 되어 메모리에서 삭제**된다. 
5. 4와 똑같은 방식으로, **b가 삭제 되었으므로 c 역시 참조카운트가 0 이 되어 메모리에서 삭제**된다. 

이처럼, 메모리에서 사용되지 않는 객체들이 도미노처럼 한꺼번에 삭제된다.  
하지만, 참조 카운트 방식으로는 **순환참조** 문제를 해결할 수 없는데, 이에 대해 살펴보도록 하자.

### 17.3 순환 참조(Circular Reference)
![image](../assets/%EA%B7%B8%EB%A6%BC%2017-2.png)
```python
class obj:
    pass

a = obj() 
b = obj()
c = obj()

a.b = b 
b.c = c 
c.a = a # 이전 코드와 달라진 부분, 순환참조를 위해 설정 

a=b=c=None 
```
앞서 살펴본 코드와의 차이점은 순환 참조를 설명하기 위해 `c.a=a` 코드가 추가 되었다는 점이다. 즉, **세개의 객체가 원 모양을 이루며 서로가 서로를 참조하고 있는 상황**이다.

구체적으로 살펴보면, 위의 오른쪽 그림을 살펴보면 `a=b=c=None`을 실행했지만 **세개의 객체 모두 불필요함에도 불구하고 순환참조 문제로 참조카운트를 1**을 가지고 있다.  
즉, 참조 카운트 방식으로는 메모리에서 삭제할 수 없는 상황이다. 이때 등장하는 것이 `GC(Generational Garbage Collection)` 메모리 관리 방식으로 참조 카운트 방식보다 영리한 방법으로 불필요한 객체를 찾아낸다.

`GC` 방식은 참조 카운트와 달리 **메모리가 부족해지는 시점** 에 파이썬 인터프리터에 의해 실행된다.(물론 명시적으로 `import gc ; gc.collect()` 로 실행가능)  
`GC`는 순환참조를 올바른 방식으로 처리하지만, 메모리 해체를 `GC`에 미루다 보면 추후 프로그램의 전체 메모리 사용량이 커지는 원인이 되며, 머신러닝/딥러닝과 같이 메모리 자원이 중요한 프로그램에서는 문제가 발생할 수 있다. 
따라서, 프로그램 개발시 이를 고려하여 순환참조를 만들지 않도록 코드를 작성하는 것이 중요하다. 


그렇다면 **현재의 Dezero에서는 순환 참조 문제가 발생하지 않고 있을까?**

![image](../assets/%EA%B7%B8%EB%A6%BC%2017-3.png)

위의 그림을 보면, `Function` 은 input과 output의 `Variable`을 참조하고 있고, 출력 `Variable`은 creator인 `Function`을 참조하고 있다. 즉, `Function`과 `Variable`간의 순환 참조 구조가 발생한다.  
이를 해결하기 위해서 파이썬 표준모듈인 `weakref`를 알아보자.

### 17.4 weakref 모듈
파이썬에서는 `weakref.ref` 함수를 이용해서 **다른 객체를 참조하되 참조 카운트는 증가하지 않는** `약한 참조(weak reference)`를 만들 수 있다.

```python
import weakref
import numpy as np 

a = np.array([1,2,3]) # a : 참조 카운트 1
b= weakref.ref(a)  # a : 참조 카운트 1 (약한 참조로 a의 참조 카운트는 증가하지 않는다), b : 참조 카운트 1 

b
# <weakref at 0x10e1ed720; to 'numpy.ndarray' at 0x108294f90>

b()
# [1 2 3 ]
```


In [10]:
import weakref

import numpy as np


def as_array(x):

    """
    0차원 ndarray / ndarray가 아닌 경우
    """
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    """
    Function Base Class

    """

    def __call__(self, *inputs):  #  1. * 를  활용하여 임의 개수의 인수
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # 1. 리스트 언팩
        if not isinstance(ys,tuple): # 2. 튜플이 아닌 경우 추가 지원 
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        ####################################
        #  수정 전 : self.outputs = outputs
        self.outputs = [weakref.ref(output) for output in outputs] #  약한 참조를 가리키도록 변경 
        ####################################

        # 2. 리스트의 원소가  하나라면  첫번째 원소를 반환
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        """
        구체적인 함수 계산 담당
        """
        raise NotImplementedError()

    def backward(self, gys):
        """
        역전파
        """
        raise NotImplementedError()


class Variable:
    def __init__(self, data: np.ndarray) -> None:
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")
        self.data = data
        self.grad = None  # gradient
        self.creator = None  # creator
        self.generation = 0 # 세대수를 기록하는 변수 

    def set_creator(self, func) -> None:
        self.creator = func
        self.generation = func.generation+1 # 세대를 기록한다 ( 부모세대 + 1)

    def backward(self):
        """
        자동 역전파 (반복)
        """
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()  
            ###########################################
            # 수정 전 : gys = [output.grad for output in f.outputs] # 1. 순전파의 결과가 **여러개의 출력인 경우**를 처리 
            gys = [output().grad for output in f.outputs]  # Fuction outputs의 weakref 모듈 변경으로 인한 반영
            ###########################################
            gxs = f.backward(*gys) # 2. 역전파 기준 **여러 개의 입력(=순전파의 여러 개 출력)** 을 처리.
            if not isinstance(gxs,tuple): # 3. 역전파 **결과값이 하나인 경우(=역전파의 출력이 1개인 경우)** 튜플로 변환.
                gxs = (gxs,)
            for x,gx in zip(f.inputs,gxs): # 4. **역전파 결과가 여러개의 출력인 경우** 각각 대응
                #  첫 grad를 설정시에는 `그대로` 출력하고, 
                if x.grad is None : 
                    x.grad = gx 
                # 다음 미분은 기존 미분 값에 `더해준다.`
                else :
                    ## NOTE :  in-place 연산 (x.grad+=gx) 을 하지 않는 이유는 **메모리 참조**로 원하지 않는 값 변동이 일어 날 수 있다.
                    x.grad = x.grad + gx 
                
                
                if x.creator is not None:
                    funcs.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다

class Square(Function):
    def forward(self, x):
        y= x**2 
        return y 
    def backward(self, gy):
        x = self.inputs[0].data  #  수정 전 : x= self.input.data
        gx = 2 * x * gy 
        return gx 
    
def square(x):
    return Square()(x)

### 17.5 동작 확인
 
 
![image](../assets/%EA%B7%B8%EB%A6%BC%2017-4.png)

for 문을 사용해 위의 계산 그래프를 반복 수행해보도록 하자. for문이 두번째 반복할때,  `x`와 `y`는 덮어 씌워지고, 사용자는 이전의 계산 그래프는 더이상 참조하지 않게 된다.

In [14]:
!pip install memory_profiler 
%load_ext memory_profiler 
%memit
for i in range(10):
    x = Variable(np.random.randn(10000)) # 거대한 데이터 
    y = square(square(square(x))) # 복잡한 계산 
    
# 메모리 사용량이 증가하지 않았음을 확인할 수 있다. 

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 75.28 MiB, increment: 0.00 MiB


### Additional : weakref 추가 예제

In [32]:
import sys 
import weakref
import numpy as np 

a = np.array([1,2,3]) # a의 참조 카운트 1 
b=weakref.ref(a)  # 약한 참조로 a의 참조 카운트는 늘지 않고, b 의 참조 카운트는 1 

print(f"a의 참조 카운트 1 + getrefcount()의 파라미터로 참조 카운트 1 = {sys.getrefcount(a)}")
print(f"b의 참조 카운트 1 + getrefcount()의 파라미터로 참조 카운트 1 = {sys.getrefcount(b)}")
a= None 
print(f"a는 해제 되었으므로, 0이 나와야 하지만, None을 참조하고 있는 여러 참조값으로 임의의 참조 값 출력 = {sys.getrefcount(a)}")
print(f"b는 아직 사용 중이므로 참조 카운트 1 + getrefcount()의 파라미터로 참조 카운트 1 = {sys.getrefcount(b)}")


a의 참조 카운트 1 + getrefcount()의 파라미터로 참조 카운트 1 = 2
b의 참조 카운트 1 + getrefcount()의 파라미터로 참조 카운트 1 = 2
a는 해제 되었으므로, 0이 나와야 하지만, None을 참조하고 있는 여러 참조값으로 임의의 참조 값 출력 = 40935
b는 아직 사용 중이므로 참조 카운트 1 + getrefcount()의 파라미터로 참조 카운트 1 = 2
